# Анализ логов

In [106]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType, FloatType, DateType, TimestampType

from datetime import datetime
from pyspark.sql.functions import *

!pip install pandas
import pandas as pd

!pip install pyyaml ua-parser user-agents
from user_agents import parse
# библиотека для парсинга user-agent

!pip install clickhouse_connect
import clickhouse_connect


# импорт модулей и библиотек

In [87]:
spark = SparkSession \
    .builder \
    .appName("Log analysis") \
    .config("spark.driver.bindAddress","localhost") \
    .config("spark.ui.port","4040") \
    .getOrCreate()

# создание spark сессии

In [88]:
client_hostname = spark.read.csv('C:/Users/raspa/Desktop/data_ignore/client_hostname.csv', header=True)
# csv файл с данными клиентов

In [89]:
def edit_alias_list(str):
    if str == "[Errno 1] Unknown host":
        return "unknown"
    else:
        return str[2:len(str)-2]
    
    
def edit_address_list(str):
    if str == "null":
        return "unknown"
    elif str is None:
        return "unknown"
    else:
        return str[2:len(str)-2]
    
# функция для замены пропущенных значений и редактировании строк(очищает от скобок и кавычек в строке)

In [90]:
func_alias_list = udf(edit_alias_list, StringType())
func_address_list = udf(edit_address_list, StringType())

In [91]:
client_hostname = client_hostname.withColumn("alias_list_new", func_alias_list("alias_list"))
client_hostname = client_hostname.drop("alias_list")
client_hostname = client_hostname.withColumn("address_list_new", func_address_list("address_list"))
client_hostname = client_hostname.drop("address_list")

In [92]:
client_hostname = client_hostname.withColumnRenamed("alias_list_new", "alias_list")\
       .withColumnRenamed("address_list_new", "address_list")

In [93]:
logs = pd.read_table('C:/Users/raspa/Desktop/data_ignore/access.log', engine='python', header=None, 
                   names=["ip", "datetime", "request", "code_req", "port", "user_agent"], 
                   sep=" - - \[{1}(.+)\] \"(.*?)\" (\d+) (\d+) .+ \"(.{2,}?)\".*",  index_col=False)

# загрузка главного файла с данными
# первичная обработка с помощью RexEx

In [132]:
logs.head(5)

,ip,datetime,request,code_req,port,user_agent,browser,device
0,54.36.149.41,2019-01-22 03:56:14,GET /filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%D...,200,30577,Mozilla/5.0 (compatible; AhrefsBot/6.1; +http:...,AhrefsBot,Spider
1,31.56.96.51,2019-01-22 03:56:16,GET /image/60844/productModel/200x200 HTTP/1.1,200,5667,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...,Chrome Mobile,Huawei
2,31.56.96.51,2019-01-22 03:56:16,GET /image/61474/productModel/200x200 HTTP/1.1,200,5379,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...,Chrome Mobile,Huawei
3,40.77.167.129,2019-01-22 03:56:17,GET /image/14925/productModel/100x100 HTTP/1.1,200,1696,Mozilla/5.0 (compatible; bingbot/2.0; +http://...,bingbot,Spider
4,91.99.72.15,2019-01-22 03:56:17,GET /product/31893/62100/%D8%B3%D8%B4%D9%88%D8...,200,41483,Mozilla/5.0 (Windows NT 6.2; Win64; x64; rv:16...,Firefox,Windows


In [130]:
logs.isnull().sum()

# проверка данных на пропущенные значения

ip            0
datetime      0
request       8
code_req      0
port          0
user_agent    0
browser       0
device        0
dtype: int64

In [131]:
logs.code_req = logs.code_req.fillna(0)
logs.port = logs.port.fillna(0)
logs.request = logs.request.fillna('unknown')
logs = logs.astype({"code_req": int, "port": int, "user_agent": str})


# замена пропущенных значений в столбцах на 0
# для конвертации в тип *integer*

In [171]:
format_date = '%d/%b/%Y:%H:%M:%S +0330'
format_date_now = '%Y-%m-%d %H:%M:%S'

In [174]:
logs.datetime = logs.datetime.apply(lambda x: datetime.strptime(x, format_date) if x is not(None) else datetime.strptime(str(datetime.now().replace(microsecond=0)), format_date_now))

# перевод даты из строки в формат datetime

CPU times: total: 10.7 s
Wall time: 10.9 s


In [114]:
%%time
logs["browser"] = logs.user_agent.apply(lambda x: parse(x).browser.family)

# вытащил из user-agent название браузера

CPU times: total: 10min 22s
Wall time: 10min 24s


In [117]:
%%time
logs["device"] = logs.user_agent.apply(lambda x: parse(x).device.brand if parse(x).device.brand is not(None) else parse(x).os.family)

# здесь вытащил бренд смартфона

CPU times: total: 13min 1s
Wall time: 13min 1s


In [ ]:
logs.show(5)

In [134]:
client = clickhouse_connect.get_client(host='localhost', username='default', password='0000')

# инициализация подключения к бд ClickHouse

In [176]:
client.command('CREATE DATABASE IF NOT EXISTS logs')

client.command('CREATE TABLE IF NOT EXISTS logs.log (ip String, datetime DateTime, request String, code_req Integer, port Integer, user_agent String, browser String, device String) ENGINE Memory')

# Создание БД и создание таблицы для загрузки датафрейма с логами

''

In [177]:
client.insert('logs.log', logs, column_names=["ip", "datetime", "request", "code_req", "port", "user_agent", "browser", "device"]) 

# вставка данных в БД 

In [535]:
result_2_3 = client.query("SELECT device, uniq(ip) AS count_users FROM logs.log GROUP BY device ORDER BY device").result_set
data_marts = pd.DataFrame(list(result_2_3), columns=['device_name', 'users_count'])

data_marts['percent_users'] = (data_marts['users_count']/data_marts['users_count'].sum())*100
# result_4

result_5 = client.query("SELECT device, count(request) AS cc FROM logs.log GROUP BY device ORDER BY device").result_set
result_5_df = pd.DataFrame(list(result_5), columns=['device_name', 'actions_device_count'])
data_marts = data_marts.merge(result_5_df)

data_marts['percent_actions'] = (data_marts['actions_device_count']/data_marts['actions_device_count'].sum())*100
# result_6

result_7 = client.query("SELECT browser, count(*) AS cc FROM logs.log GROUP BY browser ORDER BY cc DESC LIMIT 6").result_set
result_7_list = dict()
for device in data_marts.device_name:
    query = client.query("SELECT browser, count(*) AS cc FROM logs.log WHERE device = '"+device+"' GROUP BY browser ORDER BY cc DESC LIMIT 5").result_set
    query_dict = {}
    for y in range(len(query)):
        query_dict[query[y][0]] = query[y][1]
    result_7_list[device] = str(query_dict)
result_7_df = pd.DataFrame.from_dict(result_7_list, orient='index', columns=['top_browsers'])
result_7_df = result_7_df.reset_index(names='device_name')
data_marts = data_marts.merge(result_7_df)

result_8 = client.query("SELECT device, count(code_req) AS cc FROM logs.log WHERE code_req != 200 GROUP BY device ORDER BY device").result_set
result_8_df = pd.DataFrame(list(result_8), columns=['device_name', 'count_code_not_200'])
data_marts = data_marts.merge(result_8_df)

result_9 = client.query("SELECT device, groupArray(code_req) AS cc FROM logs.log WHERE code_req != 200 GROUP BY device ORDER BY device").result_set
result_9_list = list()
for x in range(len(result_9)):
    result_9_list.append((result_9[x][0], str(dict(pd.Series(result_9[x][1]).value_counts()))))
result_9_df = pd.DataFrame(result_9_list, columns=['device_name', 'count_code'])
data_marts = data_marts.merge(result_9_df)


# result_x = результат запроса в виде списка для пункта X в задании.

In [531]:
data_marts.to_csv('C:/Users/raspa/Desktop/data_marts_logs.csv')

In [532]:
data_marts

,device_name,users_count,percent_users,actions_device_count,percent_actions,top_browsers,count_code_not_200,count_code
0,Acer,10,0.003709,260,0.002508,"{'Chrome': 223, 'Android': 33, 'Chrome Mobile ...",8,"{302: 7, 404: 1}"
1,Alcatel,52,0.019286,975,0.009407,"{'Android': 568, 'Chrome Mobile': 279, 'Chrome...",114,"{404: 62, 302: 33, 499: 8, 403: 5, 500: 3, 304..."
2,Amazon,3,0.001113,4,0.000039,{'Amazon Silk': 4},1,{301: 1}
3,Apple,24981,9.265228,742618,7.164564,"{'Mobile Safari': 441761, 'Chrome Mobile iOS':...",99326,"{404: 69205, 302: 20520, 499: 3850, 301: 2879,..."
4,Archos,5,0.001854,66,0.000637,{'Android': 66},6,"{404: 5, 302: 1}"
5,Asus,1698,0.629773,29717,0.286701,"{'Chrome Mobile': 20691, 'Chrome': 3319, 'Chro...",1027,"{302: 527, 304: 180, 499: 137, 404: 86, 403: 7..."
6,BlackBerry,29,0.010756,545,0.005258,{'BlackBerry WebKit': 545},68,"{304: 35, 404: 21, 302: 6, 301: 3, 499: 3}"
7,Chrome OS,6,0.002225,347,0.003348,{'Chrome': 347},1,{301: 1}
8,Cubot,1,0.000371,80,0.000772,{'Chrome Mobile': 80},3,"{302: 2, 499: 1}"
9,Fedora,6,0.002225,1551,0.014964,"{'Firefox': 1293, 'Chrome': 258}",18,"{302: 8, 499: 8, 301: 2}"
